In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%ls

>Importing requisite libraries

In [ ]:
import cv2
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
%pwd

>Reading UrbanSound8K.csv (Metadata)
---
Metadata: Metadata is "data that provides information about other data". In other words, it is "data about data". 


In [ ]:
df = pd.read_csv("../input/urban-sound-8k-images/UrbanSound8K.csv")
df

>Visualizing a few images
---
To make sure it is working

In [ ]:
from matplotlib import pyplot
from matplotlib.image import imread
%matplotlib inline
for i in range(4):
    filename = '../input/urban-sound-8k-images/Images/Images/fold' + str(df["fold"][i]) + '/' + df["slice_file_name"][i] + '.jpg'
    image = imread(filename)
    pyplot.imshow(image)
    pyplot.show()

>Stacking images in the form of a numpy array and saving it to Google drive for later use. This is to save processing time.
---
Storing it as file "X.npy"

In [ ]:
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

X = []

# enumerate files in the directory
for i in tqdm(range(8732)):
    filename = '../input/urban-sound-8k-images/Images/Images/fold' + str(df["fold"][i]) + '/' + df["slice_file_name"][i] + '.jpg'
    photo = load_img(filename, target_size=(224, 224))
    # convert to numpy array
    photo = img_to_array(photo, dtype=np.float16)
    # store
    X.append(photo)
# convert to a numpy arrays
X = asarray(X)
print(X.shape)
# save the reshaped photos
save('X.npy', X)

>Creating labels and storing in array Y
---
Storing it as file "label.npy"

In [ ]:
Y = []
for i in tqdm(range(8732)):
    Y.append(df["classID"])
Y = np.asarray(Y)
Y = Y[0:1]
Y = Y.reshape(8732,)
Y.shape
save('label.npy',Y)

Making sure we are working in the required directory

In [ ]:
!ls

>Do not run the below block now. Run it only if you have closed the notebook before this and starting again.

In [ ]:
# from numpy import load
# X = load('X.npy')
# Y = load('label.npy')
# print(X.shape, Y.shape)
# print(type(X))
# print(type(Y))

>Printing label array Y to check its contents.

In [ ]:
import sys 
np.set_printoptions(threshold=sys.maxsize)
#print(Y)

>One Hot Encoding

In [ ]:
!pip install np_utils
import tensorflow as tf
from numpy import array
from numpy import argmax

# one hot encode
Y = tf.keras.utils.to_categorical(Y, num_classes=10)
#print(Y)

>80% for test and 20% for train.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

>Checking contents of RAM



In [ ]:
%whos

> Clearing RAM due to memory limitations

In [ ]:
import gc
gc.collect()
del X 
del Y 
gc.collect()

In [ ]:
%whos

In [ ]:
print("X_train shape: ", X_train.shape)
print("Y_train shape: ", Y_train.shape)
print("X_test shape: ", X_test.shape)
print("Y_test shape: ", Y_test.shape)

>Defining Model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(224, 224, 3)))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPool2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(10, activation='softmax'))

model.summary()

>Optimizer: Adam
---
>Learning Rate: 0.001
---
>Loss: Categorical Crossentropy

In [ ]:
from keras.optimizers import Adam
import keras
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

> Model Fitting
---
>Batch size: 128
---
>Epochs: 50
---
>Monitor Validation accuracy and save best model.

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
filepath = 'VGG4_best.hdf5'
checkpoint = ModelCheckpoint(filepath=filepath,monitor='val_accuracy',verbose=1,save_best_only=True,mode='max')
callbacks = [checkpoint]
history = model.fit(X_train, Y_train, batch_size=128, epochs = 50, validation_data = (X_test, Y_test), callbacks=callbacks, verbose = 1)

>Train vs Test accuracy

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Test Accuracy')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

>Training loss vs Validation loss

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

In [ ]:
gc.collect()
model = keras.models.load_model("VGG4_best.hdf5")

In [ ]:
score = model.evaluate(X_test, Y_test)
print(score)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
Y_pred = model.predict(X_test)
Y_pred=np.argmax(Y_pred, axis=1)
Y_test=np.argmax(Y_test, axis=1)
cm = confusion_matrix(Y_test, Y_pred)

In [ ]:
print(cm)